# Using hyperopt-sklearn with `Bench`


[hyperopt-sklearn](https://github.com/hyperopt/hyperopt-sklearn) is a popular library for tuning many "classical" ML models. The convenience this library brings is that it has predefined hyperparameter grids that are automatically tuned when one calls `fit`.

One difficulty is that it currently does not have a convenient interface for providing custom validation sets (see [this issue](https://github.com/hyperopt/hyperopt-sklearn/issues/152)). We will use this as one other example for how to implement a model for benchmarking with `mofdscribe`.


In [1]:
from mofdscribe.bench import LogkHCO2IDBench
from mofdscribe.datasets.core_dataset import CoREDataset
from hpsklearn import (
    HyperoptEstimator,
    gaussian_process_regressor,
    lightgbm_regression,
    power_transformer,
    standard_scaler,
    xgboost_regression,
)

from mofdscribe.splitters import HashSplitter
import numpy as np
from copy import deepcopy


In [2]:
ds = CoREDataset()

FEATURES = list(ds.available_features)

TARGET = "outputs.logKH_CO2"


2022-08-07 16:25:09.726 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:25:09.762 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104


In [7]:
class TunedXBoost:
    def __init__(self, features):
        self.model = HyperoptEstimator(regressor=xgboost_regression("mymodel"))
        self.features = features

    def tune(self, idx, y):
        tune_splitter = HashSplitter(self.ds.get_subset(idx))
        # we will now use a simple split in two parts,
        # however, you could also use a k-fold in the tune method
        models = []
        for train_idx_, valid_idx_ in tune_splitter.k_fold(5):
            train_idx = idx[train_idx_]
            valid_idx = idx[valid_idx_]

            train_x, train_y = self.ds._df.iloc[train_idx][self.features], y[train_idx_]
            valid_x, valid_y = self.ds._df.iloc[valid_idx][self.features], y[valid_idx_]

            # we concatenate train and validation data
            # but make sure to turn of shuffling and use the last fraction of the data for validation
            x = np.concatenate([train_x, valid_x])
            y = np.concatenate([train_y, valid_y])

            valid_frac = len(valid_x) / len(x)

            model = deepcopy(self.model)
            model.fit(x, y, cv_shuffle=False, n_folds=None, valid_size=valid_frac)

            models.append((model._best_loss, model, model._best_learner))

        models = sorted(models, key=lambda x: x[0])
        print(models)
        self.model = models[0][1]

    def fit(self, idx, structures, y):
        self.tune(idx, y)
        X = self.ds._df.iloc[idx][self.features]
        self.model.fit(X, y)

    def predict(self, idx, structures):
        X = self.ds._df.iloc[idx][self.features]
        pred = self.model.predict(X)
        return pred


In [8]:
model = TunedXBoost(FEATURES)


In [9]:
bench = LogkHCO2IDBench(model, name="xgboost-hyperopt", debug=True, patch_in_ds=True)


2022-08-07 16:26:45.829 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:26:45.871 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:26:50.404 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:26:50.519 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:26:50.533 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 0.01, q (0, 0.25, 0.5, 0.75, 1)


In [10]:
report = bench.bench()


2022-08-07 16:26:51.056 | DEBUG    | mofdscribe.bench.mofbench:_score:230 - K-fold round 0, 16 train points, 4 test points
2022-08-07 16:26:55.640 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:26:55.678 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:26:55.686 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 1.0, q (0, 0.25, 0.5, 0.75, 1)


100%|██████████| 10/10 [00:02<00:00,  2.98s/trial, best loss: 2.8037048404990177]
[(1.0755024369365775, hyperopt_estimator(ex_preprocs=[],
                   preprocessing=<hyperopt.pyll.base.Apply object at 0x2c941c760>,
                   regressor=<hyperopt.pyll.base.Apply object at 0x3024517c0>,
                   space=<hyperopt.pyll.base.Apply object at 0x2c941c730>), XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.6016640106786033, colsample_bynode=1,
             colsample_bytree=0.86475929367484, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=0.002206300871933877, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.3824494643174979, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_es

/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
2022-08-07 16:29:38.787 | DEBUG    | mofdscribe.bench.mofbench:_score:230 - K-fold round 1, 16 train points, 4 test points
2022-08-07 16:29:43.768 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:127 - Dropped 3227 duplicate basenames. New length 2166
2022-08-07 16:29:43.803 | DEBUG    | mofdscribe.datasets.core_dataset:__init__:133 - Dropped 62 duplicate graphs. New length 2104
2022-08-07 16:29:43.814 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 1.0, q (0, 0.25, 0.5, 0.75, 1)


100%|██████████| 10/10 [00:02<00:00,  2.08s/trial, best loss: 1.0232745216180554]
[(1.0132224741589904, hyperopt_estimator(ex_preprocs=(),
                   preprocessing=<hyperopt.pyll.base.Apply object at 0x2c941c070>,
                   regressor=<hyperopt.pyll.base.Apply object at 0x3026e5af0>,
                   space=<hyperopt.pyll.base.Apply object at 0x2c2581730>), XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.6260962792083816, colsample_bynode=1,
             colsample_bytree=0.5291409364644426, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=0.00047919374496525343, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.009977179709185626, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=11, missing=nan, monotone_constraints='()

/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
2022-08-07 16:32:22.125 | DEBUG    | mofdscribe.bench.mofbench:_score:230 - K-fold round 2, 16 train points, 4 test points
